##  Packages <img align="left" src="./images/film_strip_vertical.png"     style=" width:40px;   " >
We will use the now familiar NumPy and Tensorflow Packages.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd



In [2]:

df = pd.read_csv(r"cf_train.csv")
# students_id = cf_train['student_id']
# courses_id = cf_train.iloc[0]
# Student IDs (first column)
cf_test = pd.read_csv('cf_test.csv')

student_ids = df.iloc[:, 0].tolist()

# Course IDs (all columns except the first one)
course_ids = df.columns[1:].tolist()
# DataFrame for count of marks above zero for each course (column)
marks_above_zero_per_course = (df.iloc[:, 1:] > 0).sum(axis=0).reset_index()
marks_above_zero_per_course.columns = ['course_id', 'count_above_zero']

# DataFrame for count of marks above zero for each student (row)
marks_above_zero_per_student = (df.iloc[:, 1:] > 0).sum(axis=1).reset_index()
marks_above_zero_per_student['student_id'] = df.iloc[:, 0]
marks_above_zero_per_student = marks_above_zero_per_student[['student_id', 0]]
marks_above_zero_per_student.columns = ['student_id', 'count_above_zero']

# Marks only (drop student_id column)
Y = df.iloc[:, 1:].values
cf_test=df.iloc[:, 1:].values
# Show the first column
print("students_id column:\n", marks_above_zero_per_student)
# Show the first row

print("\courses :\n", marks_above_zero_per_course)

# Y = marks_matrix.to_numpy()
Y=Y.T
R = (Y != 0).astype(int)  # This would be correctprint(Y)
print(Y)







students_id column:
       student_id  count_above_zero
0            656                65
1            657                65
2            658                65
3            659                65
4            660                65
...          ...               ...
2423       35200                15
2424       35224                21
2425       35227                 8
2426       35228                17
2427       35260                22

[2428 rows x 2 columns]
\courses :
    course_id  count_above_zero
0      Lvl 1                 0
1      Lvl 2                 0
2      Lvl 3                 0
3      Lvl 4                 0
4   PHAC 358              1723
..       ...               ...
70     RQUE2               397
71     RQUE3               657
72     RQUE4               128
73     RQUE5              1637
74     RQUE6               257

[75 rows x 2 columns]
[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.

**Vectorized Implementation**

It is important to create a vectorized implementation to compute $J$, since it will later be called many times during optimization. The linear algebra utilized is not the focus of this series, so the implementation is provided. If you are an expert in linear algebra, feel free to create your version without referencing the code below. 

Run the code below and verify that it produces the same results as the non-vectorized version.

In [3]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [4]:

def normalizeRatings(Y, R):
    """
    Preprocess data by subtracting mean rating for every movie (every row).
    Only include real ratings R(i,j)=1.
    [Ynorm, Ymean] = normalizeRatings(Y, R) normalized Y so that each movie
    has a rating of 0 on average. Unrated moves then have a mean rating (0)
    Returns the mean rating in Ymean.
    """
    Ymean = (np.sum(Y*R,axis=1)/(np.sum(R, axis=1)+1e-12)).reshape(-1,1)
    Ynorm = Y - np.multiply(Ymean, R) 
    return(Ynorm, Ymean)


In [5]:

# Normalize the Dataset
Ynorm, Ymean = normalizeRatings(Y, R)

Let's prepare to train the model. Initialize the parameters and select the Adam optimizer.

In [6]:
#  Useful Values
num_movies, num_users = Y.shape
num_features = 20

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')
print(Y[0,0])

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

0.0


Let's now train the collaborative filtering model. This will learn the parameters $\mathbf{X}$, $\mathbf{W}$, and $\mathbf{b}$. 

The operations involved in learning $w$, $b$, and $x$ simultaneously do not fall into the typical 'layers' offered in the TensorFlow neural network package.  Consequently, the flow used in Course 2: Model, Compile(), Fit(), Predict(), are not directly applicable. Instead, we can use a custom training loop.

Recall from earlier labs the steps of gradient descent.
- repeat until convergence:
    - compute forward pass
    - compute the derivatives of the loss relative to parameters
    - update the parameters using the learning rate and the computed derivatives 
    
TensorFlow has the marvelous capability of calculating the derivatives for you. This is shown below. Within the `tf.GradientTape()` section, operations on Tensorflow Variables are tracked. When `tape.gradient()` is later called, it will return the gradient of the loss relative to the tracked variables. The gradients can then be applied to the parameters using an optimizer. 
This is a very brief introduction to a useful feature of TensorFlow and other machine learning frameworks. Further information can be found by investigating "custom training loops" within the framework of interest.
    


In [7]:
iterations = 100
lambda_ = 20
print("num_features",num_features , 
      "\n lamda",lambda_,
      "\niterations",iterations)
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

num_features 20 
 lamda 20 
iterations 100
Training loss at iteration 0: 9432754.2
Training loss at iteration 20: 3687007.9
Training loss at iteration 40: 2596534.1
Training loss at iteration 60: 2173241.0
Training loss at iteration 80: 2003865.2


<a name="6"></a>
## 6 - Recommendations
Below, we compute the ratings for all the movies and users and display the movies that are recommended. These are based on the movies and ratings entered as `my_ratings[]` above. To predict the rating of movie $i$ for user $j$, you compute $\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)}$. This can be computed for all ratings using matrix multiplication.

In [8]:
cost = cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)
R = tf.cast(R, tf.float64)

average_cost = tf.reduce_sum(cost) / tf.reduce_sum(R)
print("Average cost:", average_cost.numpy())


Average cost: 18.303188532365443


In [9]:
p = (tf.linalg.matmul(X, tf.transpose(W)) + b )
# print(Ymean)
df = p.numpy() + Ymean
p_df = pd.DataFrame(df)



In [10]:
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [ ]:

# Build DataFrames
df_true = pd.DataFrame(Y, index=course_ids, columns=student_ids)
df_pred = pd.DataFrame(df, index=course_ids, columns=student_ids)

cf_test = pd.read_csv('cf_test.csv')
# Drop the first column from cf_test (assumes it's unnamed or index column)
cf_test = cf_test.iloc[:, 1:]

# Remove rows where the 'mark' column is zero (if column is named 'mark')
if 'mark' in cf_test.columns:
    cf_test = cf_test[cf_test['mark'] != 0]


# print(cf_test.shape)

# Give the index a name for easier melting
df_true.index.name = "course_id"
df_pred.index.name = "course_id"

# Melt both into long format
df_true_long = df_true.reset_index().melt(
    id_vars="course_id", var_name="student_id", value_name="true_mark"
)

df_pred_long = df_pred.reset_index().melt(
    id_vars="course_id", var_name="student_id", value_name="predicted_mark"
)
df_pred_long['predicted_mark'] = df_pred_long['predicted_mark'].clip(lower=0, upper=100)

# Merge df_pred_long and cf_test on student_id and course_id to get predicted_mark and mark side by side
merged_df = pd.merge(
    cf_test,
    df_pred_long,
    on=["student_id", "course_id"],
    how="inner"
)

# Merge in counts next to IDs
student_counts = marks_above_zero_per_student.rename(columns={"count_above_zero": "student_count"})
course_counts = marks_above_zero_per_course.rename(columns={"count_above_zero": "course_count"})
merged_df = merged_df.merge(student_counts, on="student_id", how="left")
merged_df = merged_df.merge(course_counts, on="course_id", how="left")

# Create a new column 'difference' as the absolute difference between predicted_mark and mark
merged_df['difference'] = (merged_df['predicted_mark'] - merged_df['mark']).abs()

# Sort the DataFrame in descending order by the 'difference' column
merged_df_sorted = merged_df.sort_values(by=['difference', 'predicted_mark'], ascending=[False, False])

# Display the sorted DataFrame with counts next to IDs
display(merged_df_sorted[[
    "student_id",
    "student_count",
    "course_id",
    "course_count",
    "predicted_mark",
    "mark",
    "difference",
    "semester_number"
]])
print("Mean of difference:", merged_df_sorted["difference"].mean())
# display(df_pred_long)
# display(cf_test)
merged_df_sorted.to_csv('errors.csv', index=False)



,student_id,student_count_above_zero,course_id,course_count_above_zero,predicted_mark,mark,difference,semester_number
3505,8671,50,RQUE3,657,100.000000,23.0,77.000000,1
1956,7218,42,PHE1,1218,81.909740,9.0,72.909740,1
934,33129,12,PHE1,1218,83.033823,12.0,71.033823,1
4979,33684,16,PHBM 453,1743,82.417325,14.0,68.417325,1
1661,33684,16,PHR 209,1940,86.822341,19.0,67.822341,1
...,...,...,...,...,...,...,...,...
8173,11310,57,PHPT 931,1043,83.994411,84.0,0.005589,2
13156,24841,38,PHBM 552,1437,68.004787,68.0,0.004787,2
13343,24933,33,PHBM 654,1690,63.995395,64.0,0.004605,2
7425,21721,42,PHCC 728,1244,79.996898,80.0,0.003102,1


Mean of difference: 9.353514467064793


In [12]:
# predictions = merged_df_sorted[["student_id", "course_id", "predicted_mark"]]
# display(predictions)
# Save DataFrame to CSV with MySQL-optimized format
df_pred_long.to_csv(
    'recommendation_results.csv',
    index=False,
    sep=',',
    quoting=1,  # Quote all fields
    escapechar='\\',
    na_rep='NULL'
)

# Then use MySQL LOAD DATA INFILE command
mysql_command = """
LOAD DATA INFILE 'C:/path/to/your/recommendation_results.csv'
INTO TABLE recommendation_results
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\\n'
IGNORE 1 ROWS;
"""

